# PyTorchでGPUを利用する
Feed-forwardネットワークのコードを例に、GPUでの動かし方を説明する。

In [ ]:
import torch
import numpy as np

In [ ]:
crossEntropyLoss = torch.nn.NLLLoss() # 損失関数

In [ ]:
def accuracy(model,X,C):
    Y = model(X)
    result = np.array([np.argmax(y) for y in Y.detach().cpu().numpy()])
    answer = C.detach().cpu().numpy()
    return np.sum(np.equal(result, answer)) / C.shape[0]

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, C_train, C_test = train_test_split(X, C, test_size=0.1, random_state=0)

In [ ]:
from torch.utils.data import Dataset, DataLoader

class MNISTTrainDataSet(Dataset):
    def __len__(self):
        return X_train.shape[0]
    
    def __getitem__(self, idx):
        return X_train[idx].astype('float'), C_train[idx]

In [ ]:
mnist = MNISTTrainDataSet()
train_loader = DataLoader(dataset=mnist,batch_size=50,shuffle=True)

GPUが複数塔載されている場合、最初のGPUは cuda:0, 次は cuda:1... のように識別されている。GPUを有効にする場合、これを指定する。これまで通りCPUで計算する場合には cpu と指定する。to(device) を実行すると、モデルパラメータまたはデータがGPUに転送される。

In [ ]:
if torch.cuda.is_available():
    dev = "cuda:0"
else:
    dev = "cpu"

device = torch.device(dev)
model = model.to(device)

trainの中でも、DataLoader から渡されたデータをまずGPUに送る。

In [ ]:
def train(model, learning_rate=0.5, num_epochs=100):
    for epoch in range(num_epochs):
        for X, C in train_loader:
            X = X.to(device)
            C = C.to(device)
            X = X.float() # DataLoaderはデータをdouble型で渡すので、後の処理のためにfloat型に変換
            Y = model(X) # forward計算
            loss = crossEntropyLoss(Y, C) # 損失の計算
            W1, b1, W2, b2 = model.parameters()
            if W1.grad is not None:
                W1.grad.data.zero_()
                b1.grad.data.zero_()
            if W2.grad is not None:
                W2.grad.data.zero_()
                b2.grad.data.zero_()
            loss.backward() # backward計算
            # 勾配法
            W1.data -= learning_rate * W1.grad.data
            b1.data -= learning_rate * b1.grad.data
            W2.data -= learning_rate * W2.grad.data
            b2.data -= learning_rate * b2.grad.data

        Y = model(X_train_t)
        print('Epoch [%d/%d], Loss: %.4f' % (epoch+1, num_epochs, crossEntropyLoss(Y, C_train_t)))

In [ ]:
hidden_units = 500
model = torch.nn.Sequential()
model.add_module("linear1", torch.nn.Linear(in_features=D,out_features=hidden_units))
model.add_module("sigmoid", torch.nn.Sigmoid())
model.add_module("linear2", torch.nn.Linear(in_features=hidden_units,out_features=M))
model.add_module("softmax2", torch.nn.LogSoftmax(dim=1))

GPUの性能がわかりやすいように、バッチサイズを1000にする

In [ ]:
train_loader = DataLoader(dataset=mnist,batch_size=1000,shuffle=True)

学習用のバッチは DataLoader が用意してくれるのでこれだけでOKだが、損失の計算やテストセットでの精度の計算には訓練データもしくはテストデータの全部が必要なので、これを準備する。まず以前と同様、PyTorch の Tensor型に変換しておく。

In [ ]:
X_train_t=torch.from_numpy(X_train).float()
C_train_t=torch.from_numpy(C_train)
X_test_t=torch.from_numpy(X_test).float()
C_test_t=torch.from_numpy(C_test)

これを(あれば)GPUに送る。

In [ ]:
X_train_t = X_train_t.to(device)
C_train_t = C_train_t.to(device)

そして学習

In [ ]:
train(model,learning_rate=0.5)

In [ ]:
X_test_t = X_test_t.to(device)
C_test_t = C_test_t.to(device)
accuracy(model,X_test_t,C_test_t)